# Création de nouveaux descripteurs et tests

## Libraries

In [1]:
import os
import numpy as np
import h5py
from matplotlib import pyplot as plt

from utils import progressbar, getSound

## Fonctions pour le test des descripteurs

### Fonction pour calculer la matrice des descripteurs

In [10]:
def computeAllDescriptors(descriptor, descriptorsName, root = './SoundDatabase', verbose = True, nbSounds = 432, duration = 5):
    """
    Compute the descriptors on all the sound samples with the function descriptor given as parameter and save
    the resulting matrix in an hdf5 file with the name descriptorsName.
    The descriptor function must only take one argument: a sound sample.
    """

    persisted_descriptors = h5py.File("./persisted_data/descriptors.hdf5", "a")

    if descriptorsName in persisted_descriptors:

        if verbose:
            print("Loading descriptors from persisted file")
        descriptors = persisted_descriptors[descriptorsName][:]

    else:

        if verbose:
            print("Creating descriptors matrix and persisting it to a file")

        descriptors = [0] * nbSounds
        
        for k, f in enumerate(os.listdir(root)):

            if verbose:
                progressbar(nbSounds, k)
                
            filename = os.path.join(root, f)
            
            sound, _ = getSound(filename, duration)
            descriptors[k] = descriptor(sound)

        if verbose:
            print()

        persisted_descriptors.create_dataset(descriptorsName, data = descriptors)
    
    persisted_descriptors.close()
    
    return np.array(descriptors)

### Fonction pour afficher les valeurs propres cumulées pour un descripteur

In [11]:
def displayDescriptorEigenvalues(descriptorMatrix, nbEigenvalues, threshold):
    """
    Display the cumulative sum of the first nbEigenvalues eigenvalues for descriptorMatrix
    Display the number of eigenvalues necessary to explain threshold * 100 % of the variance.
    """

    # Get the eigenvalues
    eigenvalues, _ = np.linalg.eig(descriptorMatrix.dot(descriptorMatrix.T))
    eigenvalues = np.abs(eigenvalues)
    eigenvalues[::-1].sort()

    sum_eigenvalues = np.cumsum(eigenvalues) / np.sum(eigenvalues)

    # Display the eigenvalues
    plt.plot(sum_eigenvalues[0:nbEigenvalues])
    plt.hlines(threshold, 0, nbEigenvalues - 1, linestyles = ':')

    print(f"Number of eigenvalues necessary to explain {round(threshold * 100)} % of the variance: {np.min(np.where(sum_eigenvalues > threshold))}")


## Scalogramme "d'ordre 3"

## Moyenne + ecart-type

## Descripteur usuel non normalisé